In [1]:
import pandas as pd
import json
from tqdm.notebook import tqdm

In [47]:
with open("test.jsonl", "r", encoding="utf-8") as file:
    raw_data = file.readlines()
data = [json.loads(line) for line in raw_data]

In [48]:
test_df = pd.DataFrame(data)
test_df.head()

,id,question
0,1,Какие достопримечательности стоит увидеть в Риме?
1,2,Где находится лучший пляж в Барселоне?
2,3,Какой лучший рынок в Стамбуле?
3,4,Что попробовать в уличной еде в Бангкоке?
4,5,Какие музеи посетить в Лондоне?


## Test only LLM

In [4]:
from agent.tools.tools import llm
from langchain_core.messages import SystemMessage, HumanMessage

In [5]:
sys_prompt = "Ответь на вопрос, используя максимум 5 предложений"

In [7]:
answers = []
for _, row in tqdm(test_df.iterrows(), desc="Processing:", total=len(test_df)):
    answers.append(llm.invoke([SystemMessage(sys_prompt), HumanMessage(row["question"])]).content)

Processing::   0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
only_llm_df = test_df.copy()
only_llm_df["answers"] = answers

## Test AI-agent

In [2]:
from agent.agent import app

In [ ]:
answers = []
for _, row in tqdm(test_df.iterrows(), desc="Processing:", total=len(test_df)):
    answers.append(app.invoke({"question": row["question"]})["generation"])

In [12]:
agent_df = test_df.copy()
agent_df["answers"] = answers

## Count METEOR metric

In [17]:
from nltk.translate.meteor_score import meteor_score
import numpy as np

In [72]:
def count_metrics(df):
    all_metrics = []
    for _, row in df.iterrows():
        all_metrics.append(meteor_score([row["question"].split()], row["answers"].split()))
    return np.round(np.mean(all_metrics), 3)

In [73]:
count_metrics(agent_df)

np.float64(0.312)

In [74]:
count_metrics(only_llm_df)

np.float64(0.235)